In [1]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
sys.path.append("../analysis/utils")


import numpy as np
import scipy.stats as stats
import pandas as pd
from sklearn.cluster import AffinityPropagation
from sklearn.manifold import TSNE

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

# import drawing_utils as drawing
import importlib
import scoring

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('../')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [3]:
# set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org'

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['compositional_abstractions']
coll = db['referring_expressions']

# which iteration name should we use?
iterationName = 'ca_ref_exp_prolific_block_tower_dev'

## look up number of trials (including paired-practice)
# numTrials = 13

In [4]:
# query = coll.find({"$and":[
#                         {'trial_type': 'survey-text'},
#                         {'iterationName': iterationName}
#                         ]})
# df_survey = pd.DataFrame(query)
# list(df_survey['response'])

In [65]:
query = coll.find({"$and":[
                        {'trial_type': 'ca-label-ref-exp'},
                        {'iterationName': iterationName}
                        ]})
df_trial_all = pd.DataFrame(query)

In [137]:
# data cleaning
df_trial = df_trial_all[(df_trial_all.workerID.isin(['charles','zoe'])) &
                        (~pd.isna(df_trial_all.batch)) & # clean straggling data from development 
                        (df_trial_all.trialStartTime > 1.680631e+12) # clean straggling data from development
                       ]

In [138]:
# 12*49 = 588 expected trials
df_trial.workerID.value_counts()

charles    588
zoe        579
Name: workerID, dtype: int64

#### Missing data topup

In [ ]:
query = coll.find({"$and":[
                        {'trial_type': 'ca-label-ref-exp'},
                        {'iterationName': 'ca_ref_exp_prolific_block_tower_topup'}
                        ]})
df_trial_topup = pd.DataFrame(query)

In [153]:
# Remove incomplete data
df_trial_pruned = df_trial[~(((df_trial.workerID == "zoe") & 
         (df_trial.dyad_gameid == "5895-de8412be-4cb1-49e4-9c61-ca6a11481af8") &
         (df_trial.batch == 2.0)) |
         ((df_trial.workerID == "zoe") & 
         (df_trial.dyad_gameid == "0208-b497c845-b076-45c9-b958-a62a3ae6e65f") &
         (df_trial.batch == 3.0)))
        ]

In [155]:
# Append topup
# df_trial = 
df_trial_pruned.append(df_trial_topup)

NameError: name 'df_trial_pruned' is not defined

In [74]:
df_trial

,_id,trialStartTime,trialEndTime,trialDuration,responses,nWarnings,stimulus,levels,sceneID,leftTower,...,time_elapsed,internal_node_id,experimentName,iterationName,workerID,gameID,studyLocation,datatype,dyad_gameid,batch
21,642c609d21425443943feaf5,1.680630e+12,1.680630e+12,2184,"[{'msgNum': 0, 'message': '3rd block from left...",0,"{'blocks': [{'x': 2, 'y': 0, 'width': 2, 'heig...","[block, tower]","[C, Pi]",C,...,2188,0.0-0.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,9558-9184ecfe-a404-42e7-988e-f8120df26869,None,trial_end,3086-e9f8b532-4969-4703-852e-f6f029d281e2,1.0
22,642c60a421425443943feaf6,1.680630e+12,1.680630e+12,6387,"[{'msgNum': 0, 'message': 'red 2 from left', '...",0,"{'blocks': [{'x': 1, 'y': 1, 'width': 1, 'heig...","[block, tower]","[L, Pi]",L,...,9186,0.0-1.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,9558-9184ecfe-a404-42e7-988e-f8120df26869,None,trial_end,3086-e9f8b532-4969-4703-852e-f6f029d281e2,1.0
23,642c60a821425443943feaf7,1.680630e+12,1.680630e+12,1784,"[{'msgNum': 0, 'message': 'Start with left red...",0,"{'blocks': [{'x': 1, 'y': 1, 'width': 1, 'heig...","[block, tower]","[L, C]",L,...,1788,0.0-0.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,2086-4a0e9f48-7d21-4cad-b310-7f3f9b779137,None,trial_end,4978-4c585194-1823-46fc-989c-e6200c5d61e6,2.0
24,642c625e21425443943feaf8,1.680630e+12,1.680630e+12,3554,"[{'msgNum': 0, 'message': '3rd block from left...",0,"{'blocks': [{'x': 2, 'y': 0, 'width': 2, 'heig...","[block, tower]","[C, Pi]",C,...,3560,0.0-0.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,5247-3beed2bb-b2c0-4a40-84fc-a9fd34612ac5,None,trial_end,3086-e9f8b532-4969-4703-852e-f6f029d281e2,2.0
25,642e51f121425443943feaf9,1.680757e+12,1.680757e+12,110450,"[{'msgNum': 0, 'message': 'put a blue block th...",0,"{'blocks': [{'x': 2, 'y': 0, 'width': 2, 'heig...","[block, tower]","[C, Pi]",C,...,110458,0.0-0.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,zoe,0509-495c9837-2fda-4f87-9b77-143faf5a8d30,None,trial_end,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,643d5c3b21425443943fef83,1.681743e+12,1.681743e+12,16968,"[{'msgNum': 0, 'message': 'nice! btw dont worr...",0,"{'blocks': [{'x': 1, 'y': 1, 'width': 1, 'heig...","[block, tower]","[L, Pi]",L,...,1684100,0.0-43.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,0030-3bd3d609-4f04-4b08-b99c-88d313d2034f,None,trial_end,9727-37a58c13-7178-412c-88d0-6b8841439106,9.0
1188,643d5c4d21425443943fef84,1.681743e+12,1.681743e+12,17170,"[{'msgNum': 0, 'message': 'cool, now we've got...",0,"{'blocks': [{'x': 2, 'y': 0, 'width': 2, 'heig...","[block, tower]","[C, Pi]",C,...,1701881,0.0-44.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,0030-3bd3d609-4f04-4b08-b99c-88d313d2034f,None,trial_end,9727-37a58c13-7178-412c-88d0-6b8841439106,9.0
1189,643d5c6521425443943fef85,1.681743e+12,1.681743e+12,23088,"[{'msgNum': 0, 'message': 'alrighty, another h...",0,"{'blocks': [{'x': 1, 'y': 0, 'width': 1, 'heig...","[block, tower]","[Pi, C]",Pi,...,1725574,0.0-45.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,0030-3bd3d609-4f04-4b08-b99c-88d313d2034f,None,trial_end,9727-37a58c13-7178-412c-88d0-6b8841439106,9.0
1190,643d5c7521425443943fef86,1.681743e+12,1.681743e+12,15263,"[{'msgNum': 0, 'message': 'lol, same house as ...",0,"{'blocks': [{'x': 1, 'y': 0, 'width': 1, 'heig...","[block, tower]","[Pi, L]",Pi,...,1741453,0.0-46.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,0030-3bd3d609-4f04-4b08-b99c-88d313d2034f,None,trial_end,9727-37a58c13-7178-412c-88d0-6b8841439106,9.0


In [156]:
df_trial.groupby(['workerID','batch']).apply(len)

workerID  batch
charles   0.0      60
          1.0      60
          2.0      60
          3.0      60
          4.0      60
          5.0      60
          6.0      60
          7.0      60
          8.0      60
          9.0      48
zoe       0.0      60
          1.0      60
          2.0      56
          3.0      55
          4.0      60
          5.0      60
          6.0      60
          7.0      60
          8.0      60
          9.0      48
dtype: int64

## Wrangle dataframe into message by message

In [113]:
# df_trial['responses'] = df_trial['responses'].apply(ast.literal_eval)

In [157]:
def counts_to_series(df):
    return pd.DataFrame.from_records(df.counts)


def extract_counts(row):
    '''
    Extracts message responses from 
    '''
    df = pd.DataFrame.from_dict(row.responses)
    df.loc[:,'_id'] = row['_id']
    df = pd.concat([df, counts_to_series(df)], axis = 1)

    return df
    

In [158]:
df_ref_exp = pd.concat(list(df_trial.apply(extract_counts, axis=1))).reset_index(drop=True)
df_ref_exp_full = df_ref_exp.merge(df_trial, how='left', on='_id')

In [168]:
df_ref_exp.groupby('message_id').apply(len)

message_id
00183cfe-ceb0-9220-7984-f33f61c61ae4    2
0021b632-3246-b7d9-bb29-66398e4a295d    2
002b2f92-8dc5-7bd9-6689-ef79f8c3c461    2
002f9cc4-096b-faff-f5b7-751f497e28aa    2
004cdaf0-0ed9-1a32-4f0f-a9db4b6a3fea    2
                                       ..
ff52f3ab-afe3-e7ea-ede8-79c21541b74a    2
ff9e16ef-3a0b-9373-567f-fb07cbc96874    2
ffb59512-fa04-a706-ef8d-ff9bfae3fcee    1
ffbdfd47-dc79-72f0-1338-94b3854ca896    2
ffc573d5-fd0b-a70e-385a-f4635e4af862    2
Length: 2143, dtype: int64

In [160]:
pd.set_option('display.max_colwidth', 0)

In [162]:
n = 0

In [177]:
n += 30
df_ref_exp[['message','block','tower']][n:n+30]

,message,block,tower
120,place a blue 2 to the right of left side,1,0
121,place a red above that blues left most tile,1,0
122,place another red above that one,1,0
123,place a blue above that red going rightward,1,0
124,place a blue 1 to the left of right side,1,0
125,place another blue next to that one going leftward,1,0
126,place a red above that blues left most tile,1,0
127,place another red above that one,1,0
128,Place blue in middle of left tick. Stack 2 reds on top of the left side of the blue block,3,0
129,"Place blue block on top of the red you just placed so it looks like an elongated ""C.""\n",1,1


In [178]:
df_ref_exp_full

,msgNum,message,message_id,counts,refExps,_id,block,tower,trialStartTime,trialEndTime,...,time_elapsed,internal_node_id,experimentName,iterationName,workerID,gameID,studyLocation,datatype,dyad_gameid,batch
0,0,put a blue block three from the left.,e3e70682-c209-4cac-629f-6fbed82c07cd,"{'block': '1', 'tower': '0'}","blue block,",642e51f121425443943feaf9,1,0,1.680757e+12,1.680757e+12,...,110458,0.0-0.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,zoe,0509-495c9837-2fda-4f87-9b77-143faf5a8d30,None,trial_end,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,0.0
1,1,place a red block on top of the left side of the blue block and then another red on top of that red,f728b4fa-4248-5e3a-0a5d-2f346baa9455,"{'block': '2', 'tower': '0'}","red block, red,",642e51f121425443943feaf9,2,0,1.680757e+12,1.680757e+12,...,110458,0.0-0.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,zoe,0509-495c9837-2fda-4f87-9b77-143faf5a8d30,None,trial_end,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,0.0
2,2,Now place a blue block on top of that that lines up with the bottom blue block,eb1167b3-67a9-c378-7c65-c1e582e2e662,"{'block': '1', 'tower': '0'}","blue block,",642e51f121425443943feaf9,1,0,1.680757e+12,1.680757e+12,...,110458,0.0-0.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,zoe,0509-495c9837-2fda-4f87-9b77-143faf5a8d30,None,trial_end,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,0.0
3,3,"remove that last blue block, place it on top of the red blocks",f7c1bd87-4da5-e709-d471-3d60c8a70639,"{'block': '1', 'tower': '0'}","blue block,",642e51f121425443943feaf9,1,0,1.680757e+12,1.680757e+12,...,110458,0.0-0.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,zoe,0509-495c9837-2fda-4f87-9b77-143faf5a8d30,None,trial_end,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,0.0
4,4,place a red block on the second grid from the right of the screen\n,e443df78-9558-867f-5ba9-1faf7a024204,"{'block': '1', 'tower': '0'}","red block,",642e51f121425443943feaf9,1,0,1.680757e+12,1.680757e+12,...,110458,0.0-0.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,zoe,0509-495c9837-2fda-4f87-9b77-143faf5a8d30,None,trial_end,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,1,"perfect, and on the right we have a long C, with the base being the third and fourth from the right",80678ee3-88e0-4d93-dc5a-7bc04c831ff4,"{'block': '0', 'tower': '1'}","long C,",643d5c6521425443943fef85,0,1,1.681743e+12,1.681743e+12,...,1725574,0.0-45.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,0030-3bd3d609-4f04-4b08-b99c-88d313d2034f,None,trial_end,9727-37a58c13-7178-412c-88d0-6b8841439106,9.0
4250,0,"lol, same house as last time I guess, verticals second and fifth from the left",d34f0013-ecba-187c-570a-d5333cc312ae,"{'block': '2', 'tower': '1'}","house, verticals,",643d5c7521425443943fef86,2,1,1.681743e+12,1.681743e+12,...,1741453,0.0-46.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,0030-3bd3d609-4f04-4b08-b99c-88d313d2034f,None,trial_end,9727-37a58c13-7178-412c-88d0-6b8841439106,9.0
4251,1,"and now on the right we have an L shape, starting second space from the right",3d6c4c31-4dc0-500f-b917-a31ef683c8ec,"{'block': '0', 'tower': '1'}","L shape,",643d5c7521425443943fef86,0,1,1.681743e+12,1.681743e+12,...,1741453,0.0-46.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,0030-3bd3d609-4f04-4b08-b99c-88d313d2034f,None,trial_end,9727-37a58c13-7178-412c-88d0-6b8841439106,9.0
4252,0,"haha dont worry about it, it happens. we'll do an L with two blues starting two spaces from right",45ba13ab-6f31-1406-82e0-3b43dcd9c893,"{'block': '2', 'tower': '1'}","L, two blues,",643d5c9121425443943fef87,2,1,1.681743e+12,1.681743e+12,...,1769745,0.0-47.0,ca_referring_expressions,ca_ref_exp_prolific_block_tower_dev,charles,0030-3bd3d609-4f04-4b08-b99c-88d313d2034f,None,trial_end,9727-37a58c13-7178-412c-88d0-6b8841439106,9.0


In [45]:
df_chat = pd.read_csv('../results/csv/df_chat_cogsci21.csv')

In [46]:
df_chat

,Unnamed: 0,_id,iterationName,gameid,time,assignmentId,leftTarget,rightTarget,trialNum,turnNum,...,timeElapsedInTurn,timeElapsedInTrial,eventType,practice,individualPracticeAttempts,turnStartTime,trialStartTime,word_count,char_count,flagged
0,0,5ef78389ce2f7f58fcd0a350,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593279e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,vertical,horizontal,NaN,0,...,60567,60567,chatMessage,True,NaN,NaN,NaN,15,63,False
1,1,5ef783bace2f7f58fcd0a358,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593279e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,vertical,horizontal,NaN,0,...,20018,20018,chatMessage,True,NaN,NaN,NaN,13,70,False
2,2,5ef783e4ce2f7f58fcd0a35e,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593279e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0.0,0,...,28166,28166,chatMessage,False,NaN,NaN,NaN,9,40,False
3,3,5ef78401ce2f7f58fcd0a365,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593279e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0.0,2,...,21993,57512,chatMessage,False,NaN,NaN,NaN,23,99,False
4,4,5ef7842ace2f7f58fcd0a36e,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0.0,4,...,24789,98193,chatMessage,False,NaN,NaN,NaN,17,78,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114,31,5f590f181034572e66881404,pilot4b,9727-37a58c13-7178-412c-88d0-6b8841439106,1.599672e+12,3HL8HNGX491L01V1BSQZQM07RDF9FG,Pi,C,9.0,2,...,18109,61157,chatMessage,False,0.0,1.599672e+12,1.599672e+12,21,99,False
3115,32,5f590f3b1034572e6688140c,pilot4b,9727-37a58c13-7178-412c-88d0-6b8841439106,1.599672e+12,3HL8HNGX491L01V1BSQZQM07RDF9FG,Pi,L,10.0,0,...,16063,16063,chatMessage,False,0.0,1.599672e+12,1.599672e+12,15,78,False
3116,33,5f590f5e1034572e66881416,pilot4b,9727-37a58c13-7178-412c-88d0-6b8841439106,1.599672e+12,3HL8HNGX491L01V1BSQZQM07RDF9FG,Pi,L,10.0,2,...,19200,50916,chatMessage,False,0.0,1.599672e+12,1.599672e+12,16,77,False
3117,34,5f590fa51034572e66881422,pilot4b,9727-37a58c13-7178-412c-88d0-6b8841439106,1.599672e+12,3HL8HNGX491L01V1BSQZQM07RDF9FG,C,L,11.0,0,...,42453,42453,chatMessage,False,0.0,1.599672e+12,1.599672e+12,19,99,False


In [29]:
df_jj = pd.read_csv('../results/csv/JJ_content.csv')

In [30]:
df_jj

,gameid,time,assignmentId,leftTarget,rightTarget,trialNum,turnNum,repNum,message,timeElapsedInTurn,...,block_justin,toer_justin,scene_justin,Flagged,phrases_justin,block_julia,tower_juli,scene_juli,phrases_julia,content
0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0,0,0,put a blue block three from the left.,28166,...,1,0,0,NaN,a blue block,1.0,0,0.0,a blue block,a blue block a blue block
1,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0,2,0,place a red block on top of the left side of t...,21993,...,2,0,0,NaN,"a red block, another red",2.0,0,0.0,"a red block, another red","a red block, another red a red block, another red"
2,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,L,Pi,2,0,0,Do the same L shape on the left side as last t...,36762,...,4,1,0,NaN,"L shape, 2x blue, 2x red",4.0,1,0.0,"L shape, 2x blue, 2x red","L shape, 2x blue, 2x red L shape, 2x blue, 2x red"
3,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0,4,0,Now place a blue block on top of that that lin...,24789,...,1,0,0,NaN,a blue block,1.0,0,0.0,a blue block,a blue block a blue block
4,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0,6,0,"remove that last blue block, place it on top o...",29367,...,1,0,0,NaN,that last blue block,1.0,0,0.0,last blue block,that last blue block last blue block
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,5156-be8ae9d7-0f4f-4f69-9d5b-fa70394f9dea,1.599680e+12,3UN61F00H0PZ24YYZXDCVV9O86W5R7,C,Pi,7,0,2,"left- very tall skinny C, right - arch",13899,...,0,2,0,NaN,"very tall skinny C, arch",0.0,2,0.0,"very tall skinny C, arch","very tall skinny C, arch very tall skinny C, arch"
2139,5156-be8ae9d7-0f4f-4f69-9d5b-fa70394f9dea,1.599680e+12,3UN61F00H0PZ24YYZXDCVV9O86W5R7,L,C,8,0,2,"left- large L, right - very tall skinny C",19081,...,0,2,0,NaN,"large L, very tall skinny C",0.0,2,0.0,"large L, very tall skinny C","large L, very tall skinny C large L, very tall..."
2140,5156-be8ae9d7-0f4f-4f69-9d5b-fa70394f9dea,1.599680e+12,3UN61F00H0PZ24YYZXDCVV9O86W5R7,Pi,C,9,0,3,"left- arch, right - very tall skinny C",11690,...,0,2,0,NaN,"arch, very tall skinny C",0.0,2,0.0,"arch, very tall skinny C","arch, very tall skinny C arch, very tall skinny C"
2141,5156-be8ae9d7-0f4f-4f69-9d5b-fa70394f9dea,1.599680e+12,3UN61F00H0PZ24YYZXDCVV9O86W5R7,Pi,L,10,0,3,"you are rockin this! Left- arch, right- large L",21226,...,0,2,0,NaN,"arch, large L",0.0,2,0.0,"arch, large L","arch, large L arch, large L"
